## Group Number 1  
**Members:**  
- Ayush Bhargava (20889700)  
- Adam Stevenson (20952151)  
- Michael Stevenson (20952355)  
- Mason Au (20952355)  
- AJ Chen (20930079)

---

In [57]:
# Python3.10
%pip install pandas numpy docplex

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\User\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [58]:
import pandas as pd
import numpy as np
from docplex.mp.model import Model

# Base Model
---

## Data
---

In [59]:
raw_materials = [1, 2]
products = ['A', 'B']  # A = Chair, B = Drawer
suppliers = [1, 2, 3]
plants = ['A', 'B', 'C']
customers = [1, 2, 3]

#### Supplier Capacities

In [60]:
supplier_capacities = pd.DataFrame({'Supplier': ['Supplier 1', 'Supplier 2', 'Supplier 3'], 'Plastic Granule 1 (boxes)': [4800, 5000, 0], 'Plastic Granule 2 (boxes)': [5000, 0, 5000]})
display(supplier_capacities) # For visualization
supplier_capacities = {
    (1,1): 4800, (2,1): 5000,
    (1,2): 5000, (2,2): 0,
    (1,3): 0,    (2,3): 5000
}

,Supplier,Plastic Granule 1 (boxes),Plastic Granule 2 (boxes)
0,Supplier 1,4800,5000
1,Supplier 2,5000,0
2,Supplier 3,0,5000


#### Raw Material Costs

In [61]:
raw_material_costs = pd.DataFrame({'Supplier': ['Supplier 1', 'Supplier 2', 'Supplier 3'], 'Plastic Granule 1 (₱100s)': [3.0, 2.0, np.nan], 'Plastic Granule 2 (₱100s)': [5.0, np.nan, 4.0]})
display(raw_material_costs) # For visualization
raw_material_costs = {
    (1,1): 3, (2,1): 5,
    (1,2): 2,
    (2,3): 4
}

,Supplier,Plastic Granule 1 (₱100s),Plastic Granule 2 (₱100s)
0,Supplier 1,3.0,5.0
1,Supplier 2,2.0,NaN
2,Supplier 3,NaN,4.0


#### Plant Safety Stock

In [62]:
plant_safety_stock = pd.DataFrame({'Plant': ['Plant A', 'Plant B', 'Plant C'], 'Plastic Granule 1 (boxes)': [1200, 800, 1500], 'Plastic Granule 2 (boxes)': [1400, 2100, 1100]})
display(plant_safety_stock) # For visualization
plant_safety_stock = {
    ('A',1): 1200, ('A',2): 1400,
    ('B',1): 800,  ('B',2): 2100,
    ('C',1): 1500, ('C',2): 1100
}

,Plant,Plastic Granule 1 (boxes),Plastic Granule 2 (boxes)
0,Plant A,1200,1400
1,Plant B,800,2100
2,Plant C,1500,1100


#### Production Capacity

In [63]:
production_capacity = pd.DataFrame({'Plant': ['Plant A', 'Plant B', 'Plant C'], 'Plastic Chair': [1500, 2000, 1800], 'Plastic Drawer': [2000, 1800, 2500]})
display(production_capacity) # For visualization
production_capacity = {
    ('A','A'): 1500, ('A','B'): 2000,
    ('B','A'): 2000, ('B','B'): 1800,
    ('C','A'): 1800, ('C','B'): 2500
}

,Plant,Plastic Chair,Plastic Drawer
0,Plant A,1500,2000
1,Plant B,2000,1800
2,Plant C,1800,2500


#### Production Costs

In [64]:
production_costs = pd.DataFrame({'Plant': ['Plant A', 'Plant B', 'Plant C'], 'Plastic Chair (₱100s)': [1, 4, 2], 'Plastic Drawer (₱100s)': [2, 3, 3]})
display(production_costs) # For visualization
production_cost = {
    ('A','A'): 1, ('A','B'): 2,
    ('B','A'): 4, ('B','B'): 3,
    ('C','A'): 2, ('C','B'): 3
}

,Plant,Plastic Chair (₱100s),Plastic Drawer (₱100s)
0,Plant A,1,2
1,Plant B,4,3
2,Plant C,2,3


#### Material to Product Conversion

In [65]:
material_to_product_conversion = pd.DataFrame({'Product': ['Plastic Chair', 'Plastic Drawer'], 'Plastic Granule 1 (boxes)': [1, 2], 'Plastic Granule 2 (boxes)': [2, 1]})
display(material_to_product_conversion) # For visualization
conversion = {
    ('A',1): 1, ('A',2): 2,
    ('B',1): 2, ('B',2): 1
}

,Product,Plastic Granule 1 (boxes),Plastic Granule 2 (boxes)
0,Plastic Chair,1,2
1,Plastic Drawer,2,1


#### Customer Demands

In [66]:
customer_demands = pd.DataFrame({'Customer': ['Customer 1', 'Customer 2', 'Customer 3'], 'Plastic Chair': [500, 600, 1100], 'Plastic Drawer': [900, 1400, 1200]})
display(customer_demands) # For visualization
customer_demand = {
    (1,'A'): 500,  (1,'B'): 900,
    (2,'A'): 600,  (2,'B'): 1400,
    (3,'A'): 1100, (3,'B'): 1200
}

,Customer,Plastic Chair,Plastic Drawer
0,Customer 1,500,900
1,Customer 2,600,1400
2,Customer 3,1100,1200


#### Transportation Costs (Raw Materials)

In [67]:
transportation_costs_raw_materials = pd.DataFrame({'From': ['Supplier 1', 'Supplier 2', 'Supplier 3'], 'To Plant A (₱100s)': [7, 6, 13], 'To Plant B (₱100s)': [4, 10, 9], 'To Plant C (₱100s)': [5, 8, 11]})
display(transportation_costs_raw_materials) # For visualization
transport_cost_rm = {
    (1,'A'): 7, (1,'B'): 4, (1,'C'): 5,
    (2,'A'): 6, (2,'B'): 10, (2,'C'): 8,
    (3,'A'): 13, (3,'B'): 9, (3,'C'): 11
}

,From,To Plant A (₱100s),To Plant B (₱100s),To Plant C (₱100s)
0,Supplier 1,7,4,5
1,Supplier 2,6,10,8
2,Supplier 3,13,9,11


#### Transportation Costs (Finished Products)

In [68]:
transportation_costs_finished_products = pd.DataFrame({'From': ['Plant A', 'Plant B', 'Plant C'], 'To Customer 1 (₱100s)': [8, 12, 11], 'To Customer 2 (₱100s)': [10, 7, 9], 'To Customer 3 (₱100s)': [5, 9, 15]})
display(transportation_costs_finished_products) # For visualization
transport_cost_fg = {
    ('A',1): 8, ('A',2): 10, ('A',3): 5,
    ('B',1): 12, ('B',2): 7,  ('B',3): 9,
    ('C',1): 11, ('C',2): 9,  ('C',3): 15
}

,From,To Customer 1 (₱100s),To Customer 2 (₱100s),To Customer 3 (₱100s)
0,Plant A,8,10,5
1,Plant B,12,7,9
2,Plant C,11,9,15


## Base Model Objective Function
---

### Table 9. Parts of the Objective Function

| $Z_n$ | Equation Number | Cost Type                  |
|------|------------------|----------------------------|
| $Z_1$ | (1)             | Raw Material Purchase Cost |
| $Z_2$ | (2)             | Production Cost            |
| $Z_3$ | (3)             | Transportation Cost (Raw Metrials)        |
| $Z_4$ | (4)             | Transportation Cost (Finished Products)        |
| $Z$   | (5)             | Total Cost                 |


### Decision Variables

Let $RM_{jlm}$ be the amount of each raw material ordered from each supplier and delivered to each plant, where:
- $j$ is the raw material type [1, 2]
- $l$ is which supplier the material comes from [1, 2, 3]
- $m$ is the plant the material is sent to [A, B, C]

Let $P_{knm}$ be the amount of each finished project sent from each plant to each customer, where:
- $k$ is the product type [A, B]
- $n$ is which plant the product comes from
- $m$ is which customer the product is delivered to


In [69]:
mdl = Model(name="Plastic_Manufacturing_Optimization")

# Decision variables
RM = {(j,l,m): mdl.integer_var(name=f"RM{j}{l}{m}") for j in raw_materials for l in suppliers for m in plants}
P  = {(k,m,n): mdl.integer_var(name=f"P{k}{m}{n}") for k in products for m in plants for n in customers}

mdl.minimize(
    # Raw Material Purchase
    mdl.sum(raw_material_costs.get((j,l), 0) * RM[j,l,m] for j in raw_materials for l in suppliers for m in plants) +

    # Production Cost
    mdl.sum(production_cost[m,k] * P[k,m,n] for k in products for m in plants for n in customers) +

    # Transportation Costs - Raw Materials
    mdl.sum(transport_cost_rm[l,m] * RM[j,l,m] for j in raw_materials for l in suppliers for m in plants) +

    # Transportation Costs - Finished Goods
    mdl.sum(transport_cost_fg[m,n] * P[k,m,n] for k in products for m in plants for n in customers)
)

## Base Constraints
---

**Supplier Capacity** : ```Ensures the company will not order an amount greater than what is available```

In [70]:
for j in raw_materials:
    for l in suppliers:
        mdl.add_constraint(mdl.sum(RM[j,l,m] for m in plants) <= supplier_capacities.get((j,l), 0))

**Plant Safety Stock** : ```Ensures
that the plants of the company will have an adequate level of
safety stock```

In [71]:
for m in plants:
    for j in raw_materials:
        mdl.add_constraint(mdl.sum(RM[j,l,m] for l in suppliers) >= plant_safety_stock[m,j])

**Production Capacity** : ```Ensures that the amount of
products produced by the plants will not go beyond the
production limit```

In [72]:
for m in plants:
    for k in products:
        mdl.add_constraint(mdl.sum(P[k,m,n] for n in customers) <= production_capacity[m,k])

**Raw Material Conversion** : ```Ensures that
there is an adequate number of raw materials for the
production of the finished goods```

In [73]:
for m in plants:
    for j in raw_materials:
        # LHS: all incoming RM of type j to plant m
        lhs = mdl.sum(RM[j,l,m] for l in suppliers)

        # RHS: usage from products
        rhs = mdl.sum(conversion[k,j] * P[k,m,n] for k in products for n in customers)
        mdl.add_constraint(lhs >= rhs)

**Customer Demand** : ```Ensures that stock outs will be avoided and that service level will be at full if there is indeed a solution to
the model```

In [74]:
for n in customers:
    for k in products:
        mdl.add_constraint(mdl.sum(P[k,m,n] for m in plants) >= customer_demand[n,k])

## Base Model Solution

---

In [75]:
solution = mdl.solve(log_output=True)
if solution:
    print("Objective value (Total Cost):", solution.objective_value)
    for var in mdl.iter_integer_vars():
        if var.solution_value > 0:
            print(f"{var.name} = {int(var.solution_value)}")
else:
    print("No solution found.")

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 2 rows and 6 columns.
Reduced MIP has 28 rows, 30 columns, and 108 nonzeros.
Reduced MIP has 0 binaries, 30 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Found incumbent of value 269014.000000 after 0.02 sec. (0.17 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 28 rows, 30 columns, and 108 nonzeros.
Reduced MIP has 0 binaries, 30 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0.00 sec. (0.07 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                       269014.0000   

 **Model Formulation seems correct** - results are identical to the ones presented in the paper:
- ₱21,940,000 total cost
- ₱1,675,400 in savings

# Extended Model
---
This extension adds carbon emission penalties for CO₂ emissions above the allowed threshold for the permit (different threshold per plant, based on chacteristics of the plant like size, location, etc.). 

Let’s say:

- Each unit of product $k$ (chair/drawer) produced at plant $m$ emits $e_{km}$ units of CO₂.

- Each plant $m$ has an emissions penalty threshold: $E_m$.

- Each plant $m$ has a permit price of $PP_m$

The assumptions are:

- $e_{km}$, $PP_m$ and $E_m$ are known
- Every unit of CO₂ emitted over the threshold will be penalized at the same known rate $PR$ (say ₱1000 / Unit CO₂) regardless of the plant / amount over threshold

## Extended Data
---

### Emissions ($e_{km}$)

In [76]:
penalty_per_unit = 10

e_km = {
    ('A', 'A'): 1.5,   # Plant A, Chair (A)
    ('A', 'B'): 2.0,   # Plant A, Drawer (B)
    ('B', 'A'): 1.8,
    ('B', 'B'): 2.1,
    ('C', 'A'): 1.4,
    ('C', 'B'): 1.9
}

display(pd.DataFrame(
    [(plant, product, emission) for (plant, product), emission in e_km.items()],
    columns=pd.Index(['Plant', 'Product', 'Emission'])
))

,Plant,Product,Emission
0,A,A,1.5
1,A,B,2.0
2,B,A,1.8
3,B,B,2.1
4,C,A,1.4
5,C,B,1.9


### Emission Thresholds ($E_m$)

In [77]:
E_m = {
    'A': 1000,
    'B': 1200,
    'C': 1450
}

display(pd.DataFrame(
    list(E_m.items()),
    columns=pd.Index(['Plant', 'Emission Threshold'])
))

,Plant,Emission Threshold
0,A,1000
1,B,1200
2,C,1450


### Permit Prices ($PP_m$)

In [78]:
PP_m = {
    'A': 30000,
    'B': 36000,
    'C': 43500
}

display(pd.DataFrame(
    list(PP_m.items()),
    columns=pd.Index(['Plant', 'Permit Price'])
))

,Plant,Permit Price
0,A,30000
1,B,36000
2,C,43500


## Extended Objective Function
---

### Table 9. New term(s) in objective function

| $Z_n$ | Equation Number | Cost Type                  |
|------|------------------|----------------------------|
| $Z_1$ | (1)             | Raw Material Purchase Cost |
| $Z_2$ | (2)             | Production Cost            |
| $Z_3$ | (3)             | Transportation Cost (Raw Metrials)        |
| $Z_4$ | (4)             | Transportation Cost (Finished Products)        |
| $Z_5$ | (5)             | **Emission Penalty**        |
| $Z_6$ | (6)             | **Permit Cost**        |
| $Z$   | (7)             | Total Cost                 |


### New Decision Variable(s)

Let $S_m$ be the excess emissions for each plant, where:

- $m$ is the plant [A, B, C]

Let $O_m$ be 1 if plant $m$ is opened, 0 otherwise

In [79]:
mdl = Model(name="Plastic_Manufacturing_Optimization")

# Base Decision variables
RM = {(j,l,m): mdl.integer_var(name=f"RM{j}{l}{m}") for j in raw_materials for l in suppliers for m in plants}
P  = {(k,m,n): mdl.integer_var(name=f"P{k}{m}{n}") for k in products for m in plants for n in customers}

# New Decision Variables
S = {m: mdl.continuous_var(lb=0, name=f"S_{m}") for m in plants}
O_m = {m: mdl.binary_var(name=f"O_{m}") for m in plants}

# New Objective Function
mdl.minimize(
    # Raw Material Purchase
    mdl.sum(raw_material_costs.get((j, l), 0) * RM[j, l, m] for j in raw_materials for l in suppliers for m in plants) +
    # Production Cost
    mdl.sum(production_cost[m, k] * P[k, m, n] for k in products for m in plants for n in customers) +
    # Transportation Costs - Raw Materials
    mdl.sum(transport_cost_rm[l, m] * RM[j, l, m] for j in raw_materials for l in suppliers for m in plants) +
    # Transportation Costs - Finished Goods
    mdl.sum(transport_cost_fg[m, n] * P[k, m, n] for k in products for m in plants for n in customers) +
    # Emission Penalty
    penalty_per_unit * mdl.sum(S[m] for m in plants) +
    # Permit Cost
    mdl.sum(PP_m[m] * O_m[m] for m in plants)
)

## Extended Constraint(s)
---

In [80]:
# Existing constraints

# Supplier Capacity
for j in raw_materials:
    for l in suppliers:
        mdl.add_constraint(mdl.sum(RM[j,l,m] for m in plants) <= supplier_capacities.get((j,l), 0))

# Production Capacity
for m in plants:
    for k in products:
        mdl.add_constraint(mdl.sum(P[k,m,n] for n in customers) <= production_capacity[m,k])

# Raw Material Conversion
for m in plants:
    for j in raw_materials:
        # LHS: all incoming RM of type j to plant m
        lhs = mdl.sum(RM[j,l,m] for l in suppliers)

        # RHS: usage from products
        rhs = mdl.sum(conversion[k,j] * P[k,m,n] for k in products for n in customers)
        mdl.add_constraint(lhs >= rhs)

# Customer Demand
for n in customers:
    for k in products:
        mdl.add_constraint(mdl.sum(P[k,m,n] for m in plants) >= customer_demand[n,k])

**Surplus emission constraint** : ```Ensures that emissions above each plant's threshold are tracked and penalized in the objective function```

*Actual Emissions - Emission Cap ≤ $S_m$*

- The excess emissions must be at least equal to the to how much you went over the limit
- It forces $S_m$ to capture the true excess amount

In [81]:
for m in plants:
    mdl.add_constraint(
        mdl.sum(
            e_km[(m, k)] * mdl.sum(P[k, m, n] for n in customers)
            for k in products
        ) - E_m[m] <= S[m]
    )

**Linking $O_m$ to plant usage** : ```Forces plant open variable to be 1 if any production occurs at that plant```

- ensures $O_m$ = 1 if any production occurs at plant m, and 0 otherwise

In [82]:
# Big-M for each plant: sum of all production capacities at plant m
big_M = {m: sum(production_capacity[m, k] for k in products) for m in plants}

for m in plants:
    mdl.add_constraint(
        mdl.sum(P[k, m, n] for k in products for n in customers) <= big_M[m] * O_m[m]
    )

**Updated Safety Stock Constraint**: ``` This is conditional on the $O_m$ variable```

*The safety stock constraint in the base model forces all plants to be open*

In [83]:
for m in plants:
    for j in raw_materials:
        mdl.add_constraint(
            mdl.sum(RM[j, l, m] for l in suppliers) >= plant_safety_stock[m, j] * O_m[m]
        )

## Extended Solution
---

In [84]:
solution = mdl.solve(log_output=True)
if solution:
    print("Objective value (Total Cost):", solution.objective_value)
    
    # Print integer variables
    print("\nInteger Variables:")
    for var in mdl.iter_integer_vars():
        if var.solution_value > 0:
            print(f"{var.name} = {int(var.solution_value)}")
    
    # Print ALL continuous variables (including zero values)
    print("\nContinuous Variables:")
    for var in mdl.iter_continuous_vars():
        print(f"{var.name} = {var.solution_value}")
    
    # Print binary O_m variables
    print("\nPlant Open Decision Variables (O_m):")
    for m in plants:
        print(f"O_{m} = {int(O_m[m].solution_value)}")
else:
    print("No solution found.")

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 2 rows and 6 columns.
Reduced MIP has 34 rows, 36 columns, and 156 nonzeros.
Reduced MIP has 3 binaries, 30 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.07 ticks)
Found incumbent of value 446900.000000 after 0.00 sec. (0.19 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 34 rows, 36 columns, and 156 nonzeros.
Reduced MIP has 3 binaries, 30 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.07 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0.00 sec. (0.13 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Bes